In [15]:
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split

In [16]:
part1 = pd.read_csv('../data/better_processed/GDP_data.csv', sep=';')
part2 = pd.read_csv('../data/better_processed/education.csv', sep=';')
part3 = pd.read_csv('../data/better_processed/unemplyment_data.csv', sep=';')
to_pred = pd.read_csv('../data/processed/population_20-22_data.csv', sep=';')[['FIPS code', 'NPOPCHG2022']]


part1=pd.merge(part1, part2, on='FIPS code', suffixes=('','_y'))
part1=pd.merge(part1, part3, on='FIPS code', suffixes=('','_y'))
part1=pd.merge(part1, to_pred, on='FIPS code', suffixes=('', '_y'))
part1.drop(part1.filter(regex='_y$').columns, axis=1, inplace=True)

In [17]:
part1.drop(columns=['FIPS code', 'GeoName', 'State', 'Area name', 'Area_name'], inplace=True, axis=1)
list(part1.columns)

['Real GDP delta from 2021',
 'Real GDP 2021',
 'Chain GDP delta from 2021',
 'Chain GDP 2021',
 'Less than a high school diploma, delta from 2017',
 'High school diploma only, delta from 2017',
 "Some college or associate's degree, delta from 2017",
 "Bachelor's degree or higher, delta from 2017",
 'Percent of adults with less than a high school diploma, delta from 2017',
 'Percent of adults with a high school diploma only, delta from 2017',
 "Percent of adults completing some college or associate's degree, delta from 2017",
 "Percent of adults with a bachelor's degree or higher, delta from 2017",
 'Less than a high school diploma, 2017-21',
 'High school diploma only, 2017-21',
 "Some college or associate's degree, 2017-21",
 "Bachelor's degree or higher, 2017-21",
 'Percent of adults with less than a high school diploma, 2017-21',
 'Percent of adults with a high school diploma only, 2017-21',
 "Percent of adults completing some college or associate's degree, 2017-21",
 "Percent of a

In [18]:
X = part1.iloc[:, :-1]
y = part1.iloc[:, -1]

In [45]:
dzewo = RandomForestClassifier(n_estimators=200, max_depth=20)
X_train, X_test, Y_train, Y_test = train_test_split(X, y > 0, test_size=.3)

dzewo.fit(X_train, Y_train)

RandomForestClassifier(max_depth=20, n_estimators=200)

In [46]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score

accuracy_score(Y_test.values.reshape(-1, 1), dzewo.predict(X_test.values))

c:\Users\Filif\anaconda3\envs\torch_gpu\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


0.762472885032538

In [48]:
part1.to_csv('../data/better_processed/TO_R.csv', sep=';', index=False)

In [55]:
part1 = pd.read_csv('../data/processed/GDP_data.csv', sep=';')
part2 = pd.read_csv('../data/processed/education_data.csv', sep=';')
part3 = pd.read_csv('../data/processed/unemployment_data.csv', sep=';')
to_pred = pd.read_csv('../data/processed/population_20-22_data.csv', sep=';')[['FIPS code', 'NPOPCHG2022']]


part1=pd.merge(part1, part2, on='FIPS code', suffixes=('','_y'))
part1=pd.merge(part1, part3, on='FIPS code', suffixes=('','_y'))
part1=pd.merge(part1, to_pred, on='FIPS code', suffixes=('', '_y'))
part1.drop(part1.filter(regex='_y$').columns, axis=1, inplace=True)

In [56]:
part1.drop(columns=['FIPS code', 'GeoName', 'State', 'Area name', 'Area_name'], inplace=True, axis=1)
list(part1.columns)

['2001 Real GDP (thousands of chained 2012 dollars) ',
 '2002 Real GDP (thousands of chained 2012 dollars) ',
 '2003 Real GDP (thousands of chained 2012 dollars) ',
 '2004 Real GDP (thousands of chained 2012 dollars) ',
 '2005 Real GDP (thousands of chained 2012 dollars) ',
 '2006 Real GDP (thousands of chained 2012 dollars) ',
 '2007 Real GDP (thousands of chained 2012 dollars) ',
 '2008 Real GDP (thousands of chained 2012 dollars) ',
 '2009 Real GDP (thousands of chained 2012 dollars) ',
 '2010 Real GDP (thousands of chained 2012 dollars) ',
 '2011 Real GDP (thousands of chained 2012 dollars) ',
 '2012 Real GDP (thousands of chained 2012 dollars) ',
 '2013 Real GDP (thousands of chained 2012 dollars) ',
 '2014 Real GDP (thousands of chained 2012 dollars) ',
 '2015 Real GDP (thousands of chained 2012 dollars) ',
 '2016 Real GDP (thousands of chained 2012 dollars) ',
 '2017 Real GDP (thousands of chained 2012 dollars) ',
 '2018 Real GDP (thousands of chained 2012 dollars) ',
 '2019 Rea

In [59]:
part1 = part1.ffill()
part1 = part1.bfill()


In [60]:
X = part1.iloc[:, :-1]
y = part1.iloc[:, -1]

In [67]:
dzewo = RandomForestClassifier(n_estimators=200, max_depth=20)
X_train, X_test, Y_train, Y_test = train_test_split(X, y > 0, test_size=.3)

dzewo.fit(X_train, Y_train)

RandomForestClassifier(max_depth=20, n_estimators=200)

In [68]:
accuracy_score(Y_test.values.reshape(-1, 1), dzewo.predict(X_test.values))

c:\Users\Filif\anaconda3\envs\torch_gpu\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


0.7331887201735358